# Mouse Brain Sagittal Anterior

导入数据，基因选择

In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
import scanpy as sc
import pickle
import datapre as DP
import cupy as cp
import imb

sc.set_figure_params(facecolor = "white")
sc.settings.verbosity = 1
# 目录名称
dirname = "mouse_brain_sagittal_anterior"
# 距离阈值
NEIGHBOR_DIS = 1.0
# GPU_ID
GPU_ID = 1

In [2]:
adata = sc.datasets.visium_sge(sample_id = "V1_Mouse_Brain_Sagittal_Anterior")
adata.var_names_make_unique()
adata.var["mt"] = adata.var_names.str.startswith("mt-")
sc.pp.calculate_qc_metrics(adata, qc_vars = ["mt"], inplace = True)
# 过滤
sc.pp.filter_genes(adata, min_counts = 10)

/home/huggs/anaconda3/envs/rapids_nwt/lib/python3.8/site-packages/anndata/_core/anndata.py:1832: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [3]:
# 选择前200个空间可变基因
DP.adata2seurat(adata, dirname)
! Rscript spagene.R mouse_brain_sagittal_anterior 200
spagene = DP.find_spa(dirname + "/spagene.txt")

Loading required package: ggplot2
Loading required package: patchwork


In [4]:
DP.setup_seed(38)

数据集构建

In [5]:
train_data, test_data, _ = DP.dataset_blind(adata, spagene, NEIGHBOR_DIS, neg_size = None)
# with open(dirname + "/train_data.pkl", "wb") as file:
#     pickle.dump(train_data, file)
# with open(dirname + "/test_data.pkl", "wb") as file:
#     pickle.dump(test_data, file)

In [6]:
# 清理内存显存用的
mempool = cp.get_default_memory_pool()
pinned_mempool = cp.get_default_pinned_memory_pool()

去除边界负样本

In [7]:
with cp.cuda.Device(GPU_ID):
    pos_index, neg_index, marked_neg_index = imb.eliminate_BD_neg(train_data.feature, train_data.label, k = 20)
    mempool.free_all_blocks()
    pinned_mempool.free_all_blocks()
    marked_neg_index = cp.asnumpy(marked_neg_index)

In [8]:
train_data.pop(marked_neg_index)
train_data.mirror_copy()
train_data.get_feature()
train_data.get_label()

训练模型

In [9]:
from model import nn_model
import evaluation as eval
model = nn_model.NeuralNetworkClassifier(batch_size = 128)
model.fit(train_data.feature, train_data.label)

Valid: 100%|██████████| 776/776 [00:02<00:00, 290.92it/s, val_loss=0.085]


Validation loss decreased (inf --> 0.085020)
----------------------------------------------------------------


Valid: 100%|██████████| 776/776 [00:02<00:00, 313.40it/s, val_loss=0.0944]


EarlyStopping counter: 1 out of 7
----------------------------------------------------------------


Valid: 100%|██████████| 776/776 [00:02<00:00, 356.39it/s, val_loss=0.149]


...

Valid: 100%|██████████| 776/776 [00:01<00:00, 414.81it/s, val_loss=0.0584]


EarlyStopping counter: 6 out of 7
----------------------------------------------------------------


Valid: 100%|██████████| 776/776 [00:02<00:00, 370.46it/s, val_loss=0.113]


EarlyStopping counter: 7 out of 7
Early stopping
----------------------------------------------------------------


NeuralNetworkClassifier(batch_size=128)

模型预测

In [10]:
# 全部test_data
predprob = model.predict_proba(test_data.feature)
eval.evaluate(test_data.label, predprob, verbose = False)

{'Accuracy': 0.8529356994572077,
 'Precision': 0.014103394062702303,
 'Recall': 0.993485342019544,
 'MCC': 0.10917639892947069,
 'F1_Score': 0.02781197282633475,
 'AUC': 0.9538467838750373,
 'Average Precision': 0.024154627420763734,
 'confusion_matrix': array([[123363,  21321],
        [     2,    305]])}

In [11]:
with cp.cuda.Device(GPU_ID):
    pos_index, neg_index, marked_neg_index_test = imb.eliminate_BD_neg(test_data.feature, test_data.label, k = 20)
    mempool.free_all_blocks()
    pinned_mempool.free_all_blocks()
    marked_neg_index_test = cp.asnumpy(marked_neg_index_test)

In [12]:
test_data.pop(marked_neg_index_test)
# test_data.mirror_copy()   # 这一步是复制正样本，将正样本两个spot的基因表达交换
test_data.get_feature()
test_data.get_label()

In [13]:
# test_data(去除边界负样本)
predprob = model.predict_proba(test_data.feature)
eval.evaluate(test_data.label, predprob, verbose = True)

{'Accuracy': 0.9983131005813778,
 'Precision': 0.9214501510574018,
 'Recall': 0.993485342019544,
 'MCC': 0.955957481513567,
 'F1_Score': 0.9561128526645768,
 'AUC': 0.9993429208752871,
 'Average Precision': 0.9335617188715541,
 'confusion_matrix': array([[32531,    52],
        [    4,   610]])}